In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
from dateutil.relativedelta import relativedelta
from xbbg import blp

def eco_analysis(eco_print='NFP TCH Index', days_back=365, set_own_std=None, asset='USDJPY Curncy'):

    eco_tickers = [eco_print]
    eco_fields = ['ACTUAL_RELEASE', 'BN_SURVEY_MEDIAN', 'FORECAST_STANDARD_DEVIATION', 'ECO_RELEASE_DT']
    start = (datetime.today() - relativedelta(day=days_back)).strftime('%Y-%m-%d')

    eco_df = blp.bdh(
        tickers = eco_tickers,
        flds = eco_fields,
        start_date=start
    )

    eco_df.columns = [' '.join(col).strip() for col in eco_df.columns.values]

    x = eco_tickers[0]

    if set_own_std:
        eco_df[f'{x} Surprise'] = (eco_df[f'{x} ACTUAL_RELEASE'] - eco_df[f'{x} BN_SURVEY_MEDIAN']).round(3)
        eco_df[f'{x} S.D Surprise'] = (eco_df[f'{x} Surprise'] / set_own_std).round(3)
        eco_df[f'{x} ECO_RELEASE_DT'] = pd.to_datetime(eco_df[f'{x} ECO_RELEASE_DT'].astype(int).astype(str), format='%Y%m%d')
    else:
        eco_df[f'{x} Surprise'] = (eco_df[f'{x} ACTUAL_RELEASE'] - eco_df[f'{x} BN_SURVEY_MEDIAN']).round(3)
        eco_df[f'{x} S.D Surprise'] = (eco_df[f'{x} Surprise'] / eco_df[f'{x} FORECAST_STANDARD_DEVIATION']).round(3)
        eco_df[f'{x} ECO_RELEASE_DT'] = pd.to_datetime(eco_df[f'{x} ECO_RELEASE_DT'].astype(int).astype(str), format='%Y%m%d')

    eco_df.index = pd.to_datetime(eco_df.index)
    hover_text = eco_df.index.strftime('%Y-%m-%d')

    above1 = (eco_df[f'{x} S.D Surprise'] > 1).sum()
    between_0_1 = ((eco_df[f'{x} S.D Surprise'] > 0).sum() & (eco_df[f'{x} S.D Surprise'] <= 1)).sum()
    between_neg1_0 = ((eco_df[f'{x} S.D Surprise'] < 0).sum() & (eco_df[f'{x} S.D Surprise'] >= -1)).sum()
    below_neg_1 = (eco_df[f'{x} S.D Surprise'] < -1).sum()

    asset_historical = blp.bdh(
        tickers = f['{asset}'],
        flds = ['PX_LAST'],
        start_date=start
    )

    asset_historical.columns = [' '.join(col).strip() for col in asset_historical.columns.values]
    asset_historical[f'{asset} 1 Day % Return'] = asset_historical[f'{asset} PX_LAST'].pct_change() * 100
    asset_historical[f'{asset} 3 Day % Return'] = asset_historical[f'{asset} PX_LAST'].pct_change(periods=3) * 100
    asset_historical[f'{asset} 3 Day % Return'] = asset_historical[f'{asset} 3 Day % Return'].shift(-3)

    eco_df = asset_historical.merge(eco_df, how='left', left_index=True, right_index=True)
    eco_df = eco_df[eco_df[f'{eco_print} ECO_RELEASE_DT'].notna()]

    summarised_eco = eco_df[[f'{eco_print} ECO_RELEASE_DT', f'{x} Surprise', f'{x} S.D Surprise', f'{asset} 1 Day % Return', f'{asset} 3 Day % Return']]

    colors = ['green' if val > 0 else 'red' for val in eco_df[f'{x} Surprise']]

    eco_analysis_fig = make_subplots(
        rows=3, cols=1,
        row_heights=[0.2, 0.25, 0.55],
        vertical_spacing=0.1,
        specs=[[{'type': 'scatter'}], [{'type': 'table'}, [{'type': 'table'}]]]
    )

    eco_analysis_fig.add_trace(go.Scatter(
            x=eco_df[f'{x} S.D Surprise'],
            y=[0]*len(eco_df),
            mode='markers',
            marker=dict(color=colors, size=10),
            text=hover_text,
            hover_template='Date: %{text}<br>S.D Surprise: %{x}<extra></extra>'
        ),
        row=1,
        col=1
    )

    eco_analysis_fig.add_trace(go.Table(
        header=dict(
            values=['S.D Surprise Range', 'Count'],
            fill_color='darkslategray',
            font=dict(color='white', size=14),
            align='center'
        ),
        cells=dict(
            values=[
                ['Above 1', 'Between 0 and 1', 'Between -1 and 0', 'Below -1'],
                [above1, between_0_1, between_neg1_0, below_neg_1]
            ],
            fill_color='black',
            font=dict(color='white', size=13),
            align='center'
        )
    ), row=2, col=1)

    sd_colors = ['green' if val > 0 else 'red' if val < 0 else 'grey' for val in eco_df[f'{x} S.D Surprise']]
    one_day_colors = ['green' if val > 0 else 'red' if val < 0 else 'grey' for val in eco_df[f'{x} 1 Day % Return']]
    three_day_colors = ['green' if val > 0 else 'red' if val < 0 else 'grey' for val in eco_df[f'{x} 3 Day % Return']]

    eco_analysis_fig.add_trace(go.Table(
        header=dict(
            values=list(summarised_eco.columns),
            fill_color='darkslategray',
            font=dict(color='white', size=12),
            align='left'
        ),
        cells=dict(
            values=[
                summarised_eco[f'{x} ECO_RELEASE_DT'].dt.strftime('%Y-%m-%d'),
                summarised_eco[f'{x} Surprise'].tolist(),
                summarised_eco[f'{x} S.D Surprise'].tolist(),
                summarised_eco[f'{x} 1 Day % Return'].tolist(),
                summarised_eco[f'{x} 3 Day % Return'].tolist()
            ],
            fill_color=[
                ['black'] * len(summarised_eco),
                sd_colors,
                sd_colors,
                one_day_colors,
                three_day_colors
            ],
            font=dict(color='white', size=11),
            align='left'
        )
    ), row=3, col=1)

    eco_analysis_fig.update_layout(
        title=dict(
            text=f'{asset} Price Action post {eco_print} | Last {days_back} days',
            x=0.5,
            xanchor='center'
        ),
        template='plotly_dark',
        paper_bgcolor='black',
        plot_bgcolor='black',
        font=dict(color='white'),
        height=850,
        annotations=[
            dict(
                text='Summary Table',
                x=0.5,
                y=0.78,
                xref='paper',
                yref='paper',
                showarrow='False',
                font=dict(size=14, color='white')
            ),
            dict(
                text='Print Surprise Table',
                x=0.5,
                y=0.48,
                xref='paper',
                yref='paper',
                showarrow=False,
                font=dict(size=14, color='white')
            )
        ]
    )

    eco_analysis_fig.show()

    return eco_df

df = eco_analysis(
    eco_print='NFP TCH Index',
    days_back=365,
    asset='USGG10YR Index'
)